In [ ]:
%run vis/xAPISG-noDataToFillVisualization.ipynb # notebook to create the visualisation with a message NoDataToFill
%run vis/helpersFunctions/barCharts.ipynb
%run vis/helpersFunctions/OrderGraphValuesAndSelectDataByXaxis.ipynb

***vis_alternatives_selected_question* function**:
* Input: players_info dictionary 
* Displays bar chart showing the alternatives selcted in each multiple-choice question

In [ ]:
outputAlternativeQuestion=widgets.Output()
def vis_alternatives_selected_question ():
    out=interactive(alternatives_selected_question,
                       #constructing a dropdown to order for alphabetic keys or for values
                        order=widgets.Dropdown(options=[('alphabetic',0),('value from highest to lowest',1),('value from lowest to highest',-1)],
                                description='Order',
                                disabled=False))
    md_text=widgets.HTML("<h4>Bar chart per question, with number of times each alternative has been selected</h4>")
    with outputAlternativeQuestion:
        print("Updating visualisation with the current selection of players and alternatives...")
    return widgets.VBox([md_text,out, outputAlternativeQuestion])

def alternatives_selected_question(order):
    #get all players selected
    playersSelected=checkboxesPlayersSelected.get_selected_options()
    alternatives_list_selected=checkboxesAlternativeSelected.get_selected_options()
    # responses per alternative
    selected_per_alternative = {}
    total_max=[]
    i=0
    for alt in alternatives_list_selected:
        selected = []
        total_max.insert(i, 0)
        for player in playersSelected:
            if alt in players_info[player]["alternatives"].keys():
                alternative, _ = zip(*players_info[player]["alternatives"][alt])
                alternative=list(alternative)
                selected.extend(alternative)
        selected_per_alternative[alt] = Counter(selected)
        for res in selected_per_alternative[alt].values():
            if res>total_max[i]:
                total_max[i]=res
        total_max[i]+=0.5
        i+=1
        
    #Good answer per alternative
    goodAnswer={}
    for player in players_info.keys():
        for alt in players_info[player]["alternatives"].keys():
            if not alt in goodAnswer.keys():
                goodAnswer[alt]=[]
            for (answer, success) in players_info[player]["alternatives"][alt]:
                if success in ['True', True]:
                    goodAnswer[alt].append(answer)
    
    # Multiple bar chart showing the alternatives selected in each multiple-choice question.
    number_of_plots = len(selected_per_alternative)
    selected_per_alternative = orderDictByDictValues(selected_per_alternative,order)
    if number_of_plots>1:
        fig, axs = plt.subplots(number_of_plots)
        i=0
        for key in selected_per_alternative.keys():
            axs[i].bar(range(len(selected_per_alternative[key])), 
                       selected_per_alternative[key].values(), 
                       tick_label=list(selected_per_alternative[key].keys()))
            axs[i].set_title('Responses selected to question '+key)
            axs[i].set_ylim(0,total_max[i])
            axs[i].set_yticks(np.arange(0, total_max[i], 1))
            j=0
            for answer in selected_per_alternative[key].keys():
                if answer in goodAnswer[key]:
                    axs[i].get_children()[j].set_color('g')
                else:
                    axs[i].get_children()[j].set_color('r')
                j+=1
            axs[i].set(xlabel="Responses selected", ylabel="Number of responses")
            i += 1
    else: 
        if number_of_plots==1:
            for key in selected_per_alternative.keys(): #only one key
                barlist=plt.bar(range(len(selected_per_alternative[key])), 
                       selected_per_alternative[key].values(), 
                       tick_label=list(selected_per_alternative[key].keys()))
                plt.title('Responses selected to question '+key)
                j=0
                for answer in selected_per_alternative[key].keys():
                    if answer in goodAnswer[key]:
                        barlist[j].set_color('g')
                    else:
                        barlist[j].set_color('r')
                    j+=1
            plt.yticks(np.arange(0, total_max[0], 1))
            plt.ylim(0,total_max[0])
        else:
            noDataToFillVis(100)
            plt.title("Responses selected to question")
        plt.xlabel("Responses selected")
        plt.ylabel("Number of responses")
    plt.tight_layout()
    plt.show()
    with outputAlternativeQuestion:
        clear_output(wait=False)